In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import spotify_download as sd
import os
import shutil
import glob
import db_util
import pandas as pd
import embedding_generation
from PIL import Image
from sklearn.decomposition import PCA
import cv2
from skimage.segmentation import slic
from skimage.color import label2rgb
import tqdm
import vtracer
from FLAS import assign_positions_to_images
import soundfile as sf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Successfully logged in


## Download Music

In [7]:
playlists = [
    "37i9dQZF1DX4o1oenSJRJd", # All Out 2000s
    "37i9dQZF1DX5Ejj0EkURtP", # All Out 2010s
    "37i9dQZF1DX2M1RktxUUHG", # All Out 2020s
    "1f6ze3bR0LdhO7zp3V56J5" # Explode by Big Freedia
]

# find path for spotdl executable
spotdl_path = !which spotdl
spotdl_path = spotdl_path[0]

tracks = []
print(f"Downloading {len(playlists)} playlists...")
if not os.path.exists("tracks"):
    os.mkdir("tracks")
for playlist in playlists:
    tracks.extend(list(sd.download_playlist(playlist, "tracks", spotdl_path)))

print("Building database...")
sd.build_db(tracks_dir="tracks")

100%|██████████| 1/1 [00:13<00:00, 13.50s/it]


Building database...


100%|██████████| 451/451 [01:45<00:00,  4.28it/s]


## Generate Audio Embeddings

In [41]:
tracks_df = db_util.get_tracks()
embedding_generation.embed_tracks(tracks_df)

0it [00:00, ?it/s]


## Generate Images

In [42]:
img_resolution = (128, 128)
source_img_dir = "style_images"
max_source_imgs = 500
target_img_dir = "images"
n_pca_components = 64
blur_radius = 63

tracks = db_util.get_tracks()

source_img_paths = glob.glob(os.path.join(source_img_dir, "*.jpg"))
if len(source_img_paths) == 0:
    print("No source images found!")
elif len(source_img_paths) > max_source_imgs:
    np.random.seed(0)
    source_img_paths = np.random.choice(source_img_paths, max_source_imgs, replace=False)
source_imgs = [np.array(Image.open(path).resize(img_resolution)) for path in source_img_paths]
source_imgs = [cv2.GaussianBlur(img, (blur_radius, blur_radius), 0) for img in source_imgs]
for i in range(len(source_imgs)):
    if len(source_imgs[i].shape) == 2:
        source_imgs[i] = np.dstack([source_imgs[i]]*3)
    else:
        source_imgs[i] = source_imgs[i][:,:,:3]
X = np.array([(img if len(img.shape) == 3 else np.dstack([img,img,img])) for img in source_imgs])
X = X.reshape(len(source_imgs), -1)
pca = PCA(n_components=n_pca_components)
pca.fit(X)
principal_components = pca.components_

embeddings = np.array(tracks.embedding.tolist())
embedding_pca = PCA(n_components=n_pca_components)
embeddings_reduced = embedding_pca.fit_transform(embeddings)

embeddings_reduced = embedding_generation.optimize_spacing(embeddings_reduced)

tracks[f"embedding_{n_pca_components}"] = [embedding for embedding in embeddings_reduced]

print("Generating images (phase 1)...")
for i, track in tracks.iterrows():
    track_id = track["id"]
    embedding = track[f"embedding_{n_pca_components}"]
    image = np.dot(embedding, principal_components).reshape(*img_resolution, 3)
    image -= image.min()
    image /= image.max()
    image *= 255
    image = image.astype(np.uint8)
    plt.imsave(os.path.join("track_images", "blurry", f"{track_id}.png"), image)

print("Generating images (phase 2)...")
for i, track in tqdm.tqdm(tracks.iterrows(), total=len(tracks)):
    track_id = track["id"]
    blurry = plt.imread(f"track_images/blurry/{track_id}.png")[:, :, :3]
    # cluster the image
    labels = slic(blurry, n_segments=16, convert2lab=True, enforce_connectivity=False)
    # recreate the image
    reconstructed = label2rgb(labels, blurry, kind='avg')
    plt.imsave(f"track_images/flattened/{track_id}.png", reconstructed)
    vtracer.convert_image_to_svg_py(f"track_images/flattened/{track_id}.png", f"track_images/vectorized/{track_id}.svg")
    !inkscape -w 256 -h 256 track_images/vectorized/{track_id}.svg -o track_images/rasterized/{track_id}.png > /dev/null 2>&1

Optimizing embedding spacing...


100%|██████████| 100/100 [00:04<00:00, 22.79it/s]


Generating images (phase 1)...
Generating images (phase 2)...


100%|██████████| 451/451 [03:28<00:00,  2.17it/s]


## Sort with LAS

In [6]:
assign_positions_to_images("track_images/rasterized", feature_vector_size=8, max_images=1024)

Added 0 images to make grid shape possible
number of paths: 451
number of unique paths: 451
Grid shape: (11, 41)
Number of images: 451
.........
Sorted with LAS in 0.120 seconds
number of ids: 451
number of unique ids: 451


## Move to Viewer

In [10]:
tracks_df = db_util.get_tracks()
overwrite_clips = False

# Create 15 second clips of each track
print("Creating clips...")
for i, track in tqdm.tqdm(tracks_df.iterrows()):
    if os.path.exists(f"Viewer/track_clips/{track['id']}.mp3") and not overwrite_clips:
        continue
    audio_path = track["audio_path"]
    audio, sample_rate = sf.read(audio_path)
    samples_per_clip = sample_rate * 15
    middle = len(audio) // 2
    audio = audio[middle - samples_per_clip // 2:middle + samples_per_clip // 2]
    sf.write(f"Viewer/track_clips/{track['id']}.mp3", audio, sample_rate)
    !ffmpeg -i Viewer/track_clips/{track['id']}.mp3 -map 0:a:0 -b:a 48k Viewer/track_clips/{track['id']}.ogg > /dev/null 2>&1
    !rm Viewer/track_clips/{track['id']}.mp3

print("Copying images to viewer...")
# Copy the images to the viewer
for i, track in tracks_df.iterrows():
    shutil.copy(f"track_images/vectorized/{track['id']}.svg", f"Viewer/track_images/{track['id']}.svg")

print("Copying layout to viewer...")
# Copy the sorted layout to the viewer
gridded_ids = np.loadtxt("gridded_ids.csv", dtype=str)
nested_list = []
for line in gridded_ids:
    nested_list.append(line.split(","))

js_list_string = nested_list.__str__().replace("'", "\"")
js_list_string = "var grid_ids = " + js_list_string
with open("Viewer/data.js", "w") as f:
    f.write(js_list_string)

Creating clips...


0it [00:00, ?it/s]

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --enable-libx264 --enable

1it [00:00,  1.79it/s]

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --enable-libx264 --enable

...

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --enable-libx264 --enable

450it [02:31,  2.96it/s]

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1674566267822/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-gpl --enable-libx264 --enable

451it [02:32,  2.97it/s]

Copying images to viewer...
Copying layout to viewer...
